In [1]:
import pandas as pd       
import os 
import math 
import numpy as np
import matplotlib.pyplot as plt  
import IPython.display as ipd  # To play sound in the notebook
import librosa
import librosa.display
import time
import warnings
import pickle as pickle
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

/Users/meghana/dev/miniconda3/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/Users/meghana/dev/miniconda3/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [2]:
SAMPLE_RATE = 22050
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("speakers_all.csv")


In [4]:
df.drop(df.columns[9:12],axis = 1, inplace = True)
print(df.columns)
df.describe()

Index(['age', 'age_onset', 'birthplace', 'filename', 'native_language', 'sex',
       'speakerid', 'country', 'file_missing?'],
      dtype='object')


,age,age_onset,speakerid
count,2172.000000,2172.000000,2172.000000
mean,33.117173,8.833333,1088.449355
std,14.453039,8.451127,628.420329
min,0.000000,0.000000,1.000000
25%,22.000000,0.000000,543.750000
50%,28.000000,8.000000,1088.500000
75%,41.000000,13.000000,1632.250000
max,97.000000,86.000000,2176.000000


In [18]:
import IPython.display as ipd  # To play sound in the notebook
fname1 = 'recordings/' + 'albanian4.mp3'
ipd.Audio(fname1, rate =22050)



In [224]:
SAMPLE_RATE = 22050
fname_f = 'recordings/' + 'english385.mp3'  
y, sr = librosa.load(fname_f, sr=SAMPLE_RATE, duration = 5) # Chop audio at 5 secs... 
C = librosa.feature.chroma_cqt(y=y, sr=sr)
print(C.shape)
print(np.mean(C,axis=1))
print(np.std(C,axis=1))
print(np.hstack((np.mean(C,axis=1),np.std(C,axis=1))))
print(np.hstack((np.mean(C,axis=1),np.std(C,axis=1))).shape)

(12, 216)
[0.38341313 0.34105598 0.3280549  0.32915976 0.3347736  0.30984292
 0.30869983 0.35170298 0.40050376 0.55909603 0.78122582 0.63569021]
[0.22008449 0.24009863 0.23054113 0.22632227 0.23614225 0.22145201
 0.24625735 0.24588354 0.25223644 0.24700904 0.29275413 0.28219173]
[0.38341313 0.34105598 0.3280549  0.32915976 0.3347736  0.30984292
 0.30869983 0.35170298 0.40050376 0.55909603 0.78122582 0.63569021
 0.22008449 0.24009863 0.23054113 0.22632227 0.23614225 0.22145201
 0.24625735 0.24588354 0.25223644 0.24700904 0.29275413 0.28219173]
(24,)


In [ ]:
print(list(df["file_missing?"].unique()))

In [5]:
# def freq_based_function(f_miss, fname):
#     if not f_miss:
#         try:
#             path = 'recordings/' + fname +".mp3"
#             data, sampling_rate = librosa.load(path, sr=SAMPLE_RATE)
#             duration = librosa.get_duration(y=data, sr=sampling_rate)
#             mfcc = librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc = 5)
#             melspec = librosa.feature.melspectrogram(data, sr=sampling_rate, n_mels=128)
#             y_harmonic, y_percussive = librosa.effects.hpss(data)
#             melspec_h = librosa.feature.melspectrogram(y_harmonic, sr=sampling_rate, n_mels=128)
#             log_h = librosa.amplitude_to_db(melspec_h)
#             melspec_p = librosa.feature.melspectrogram(y_percussive, sr=sampling_rate, n_mels=128)
#             log_p = librosa.amplitude_to_db(melspec_p)
#             zero_crossing_rate = librosa.feature.zero_crossing_rate(data)[0]
#             spectral_rolloff = librosa.feature.spectral_rolloff(data)[0]
#             spectral_centroid = librosa.feature.spectral_centroid(data)[0]
#             spectral_contrast = librosa.feature.spectral_contrast(data)[0]
#             spectral_bandwidth = librosa.feature.spectral_bandwidth(data)[0]
#             chroma_cqt = librosa.feature.chroma_cqt(y=data, sr=sampling_rate)
#             save_data={"data": data,"sampling_rate":sampling_rate,"duration":duration
#                        ,"mfcc": mfcc,"melspec":melspec,"y_harmonic":y_harmonic,"y_percussive":y_percussive
#                        ,"melspec_h":melspec_h,"log_h":log_h,"melspec_p":melspec_p,"log_p":log_p
#                        ,"zero_crossing_rate":zero_crossing_rate,"spectral_rolloff":spectral_rolloff
#                        ,"spectral_centroid":spectral_centroid,"spectral_contrast":spectral_contrast
#                        ,"spectral_bandwidth":spectral_bandwidth,"chroma_cqt":chroma_cqt}
#             with open('features/'+fname+'.pkl', 'wb') as outfile:
#                 pickle.dump(save_data, outfile, pickle.HIGHEST_PROTOCOL)
# #             return (duration,mfcc,melspec,melspec_h,melspec_p,y_harmonic, y_percussive,log_h,log_p,data,sampling_rate)
#         except Exception:
#             return 

# # durations,mfccs,melspecs,melspec_hs,melspec_ps,y_harmonics,y_percussives,log_hs,log_ps,datas,sampling_rates = [],[],[],[],[],[],[],[],[],[],[]
# for _, row in df.iterrows():
#     print(_)
#     params = row[['file_missing?', 'filename']].values
#     freq_based_function(params[0],params[1])


In [9]:
start_time = time.time()

def extract_features_from_pkl(fname):
    try:
        path = 'features/' + fname +".pkl"
        extracted_data = pd.read_pickle(path)
        data, sampling_rate = extracted_data["data"], extracted_data["sampling_rate"]
        duration = extracted_data["duration"]
        mfcc = extracted_data["mfcc"]
        melspec = extracted_data["melspec"]
        y_harmonic, y_percussive = extracted_data["y_harmonic"],extracted_data["y_percussive"]
        melspec_h = extracted_data["melspec_h"]
        log_h = extracted_data["log_h"]
        melspec_p = extracted_data["melspec_p"]
        log_p = extracted_data["log_p"]
        zero_crossing_rate = extracted_data["zero_crossing_rate"]
        spectral_rolloff = extracted_data["spectral_rolloff"]
        spectral_centroid = extracted_data["spectral_centroid"]
        spectral_contrast = extracted_data["spectral_contrast"]
        spectral_bandwidth = extracted_data["spectral_bandwidth"]
        chroma_cqt = extracted_data["chroma_cqt"]
        return (duration,mfcc,melspec,melspec_h,melspec_p,y_harmonic, y_percussive,log_h
                ,log_p,data,sampling_rate,zero_crossing_rate,spectral_rolloff,spectral_centroid
                ,spectral_contrast,spectral_bandwidth,chroma_cqt)
    except Exception:
        return 

durations,mfccs,melspecs,melspec_hs,melspec_ps,y_harmonics,y_percussives = [],[],[],[],[],[],[]
log_hs,log_ps,datas,sampling_rates = [],[],[],[]
zero_crossing_rates,spectral_rolloffs, spectral_centroids = [],[],[]
spectral_contrasts,spectral_bandwidths,chroma_cqts = [],[],[]
for _, row in df.iterrows():
    print(_)
    filename = row['filename']
    val = extract_features_from_pkl(filename)
    if val:
        durations.append(val[0])
        mfccs.append(val[1])
        melspecs.append(val[2])
        melspec_hs.append(val[3])
        melspec_ps.append(val[4])
        y_harmonics.append(val[5])
        y_percussives.append(val[6])
        log_hs.append(val[7])
        log_ps.append(val[8])
        datas.append(val[9])
        sampling_rates.append(val[10])
        zero_crossing_rates.append(val[11])
        spectral_rolloffs.append(val[12])
        spectral_centroids.append(val[13])
        spectral_contrasts.append(val[14])
        spectral_bandwidths.append(val[15])
        chroma_cqts.append(val[16])
    else:
        durations.append(None)
        mfccs.append(None)
        melspecs.append(None)
        melspec_hs.append(None)
        melspec_ps.append(None)
        y_harmonics.append(None)
        y_percussives.append(None)
        log_hs.append(None)
        log_ps.append(None)
        datas.append(None)
        sampling_rates.append(None)
        zero_crossing_rates.append(None)
        spectral_rolloffs.append(None)
        spectral_centroids.append(None)
        spectral_contrasts.append(None)
        spectral_bandwidths.append(None)
        chroma_cqts.append(None)

print("time taken "+str((time.time() - start_time)/60))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065


In [10]:
df["duration"] = durations
df["mfcc"] = mfccs
df["melspec"] = melspecs
df["melspec_h"] = melspec_hs
df["melspec_p"] = melspec_ps
df["y_harmonic"] = y_harmonics
df["y_percussive"] = y_percussives
df["melspec_harmonic"] = log_hs
df["melspec_percussive"] = log_ps
df["librosa_data"] = datas
df["librosa_sampling_rate"] = sampling_rates
df["zero_crossing_rate"] = zero_crossing_rates
df["spectral_rolloff"] = spectral_rolloffs
df["spectral_centroid"] = spectral_centroids
df["spectral_contrast"] = spectral_contrasts
df["spectral_bandwidth"] = spectral_bandwidths
df["chroma_cqt"] = chroma_cqts

In [11]:
df = df.dropna()
df.head()

,age,age_onset,birthplace,filename,native_language,sex,speakerid,country,file_missing?,duration,...,melspec_harmonic,melspec_percussive,librosa_data,librosa_sampling_rate,zero_crossing_rate,spectral_rolloff,spectral_centroid,spectral_contrast,spectral_bandwidth,chroma_cqt
32,27.0,9.0,"virginia, south africa",afrikaans1,afrikaans,female,1,south africa,False,20.772290,...,"[[-5.995369, -5.995369, -5.995369, -5.995369, ...","[[-30.363499, -29.586853, -28.244614, -23.9563...","[0.0071707093, 0.010596839, 0.00808429, 0.0093...",22050.0,"[0.01611328125, 0.0244140625, 0.033203125, 0.0...","[2950.048828125, 2971.58203125, 3068.481445312...","[1530.4223975774694, 1423.8069150707834, 1320....","[36.50567083862842, 14.012752078187578, 10.362...","[2463.8000385430314, 2273.238307315652, 2083.8...","[[0.8212306011679301, 0.8040046693991846, 0.78..."
33,40.0,5.0,"pretoria, south africa",afrikaans2,afrikaans,male,2,south africa,False,21.961497,...,"[[-24.149624, -24.149624, -24.149624, -24.1496...","[[-24.416653, -24.416653, -24.416653, -24.4166...","[-1.8169361e-05, -2.4786286e-05, -8.806779e-06...",22050.0,"[0.025390625, 0.041015625, 0.05029296875, 0.04...","[8990.1123046875, 9108.544921875, 9248.5107421...","[5123.089311931954, 5264.551708269, 5429.44865...","[32.56676585595034, 28.16572518059867, 24.7029...","[3348.660970144079, 3348.077664928403, 3389.43...","[[0.9168646531128495, 0.6017439302030426, 1.0,..."
34,43.0,4.0,"pretoria, transvaal, south africa",afrikaans3,afrikaans,male,418,south africa,False,26.880000,...,"[[-17.287262, -17.287262, -17.287262, -17.2872...","[[-15.318008, -15.318008, -15.318008, -15.3180...","[0.0019851814, 0.0027917789, 0.0013547958, 0.0...",22050.0,"[0.0244140625, 0.0322265625, 0.05126953125, 0....","[3079.248046875, 2627.05078125, 2616.284179687...","[1465.0935000653856, 1361.8608658509977, 1347....","[22.474867417512268, 26.549838842792123, 29.65...","[2159.5564308533862, 2036.7410293218215, 1875....","[[0.9405656123995829, 0.8434395912814198, 0.93..."
35,26.0,8.0,"pretoria, south africa",afrikaans4,afrikaans,male,1159,south africa,False,23.471202,...,"[[-18.92363, -18.92363, -18.92363, -18.92363, ...","[[-17.007862, -17.007862, -17.007862, -17.0078...","[0.00025531906, 0.0003802524, 0.00032837203, 0...",22050.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[8182.6171875, 8322.5830078125, 8268.75, 7967....","[2836.1443828141787, 2981.4239632758927, 2943....","[31.146817865859134, 33.83682462665064, 35.432...","[3677.497426953188, 3705.7593851497895, 3685.5...","[[0.43540816224779805, 0.43058722215309736, 0...."
36,19.0,6.0,"cape town, south africa",afrikaans5,afrikaans,male,1432,south africa,False,20.252426,...,"[[-7.7523117, 4.3265815, 9.7880745, 8.180036, ...","[[-30.512177, -17.553995, -20.826927, -35.1874...","[1.7647135e-05, -1.7076701e-06, -9.886354e-06,...",22050.0,"[0.0126953125, 0.01513671875, 0.0166015625, 0....","[1668.8232421875, 1604.2236328125, 1701.123046...","[808.4449193168485, 717.8543169790694, 817.630...","[20.726607494551942, 19.790129878017588, 21.93...","[1641.2152764813704, 1536.646836041162, 1601.9...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,..."


In [12]:
df.drop(columns=['age_onset','filename','speakerid','file_missing?'],inplace=True)
df.head()

,age,birthplace,native_language,sex,country,duration,mfcc,melspec,melspec_h,melspec_p,...,melspec_harmonic,melspec_percussive,librosa_data,librosa_sampling_rate,zero_crossing_rate,spectral_rolloff,spectral_centroid,spectral_contrast,spectral_bandwidth,chroma_cqt
32,27.0,"virginia, south africa",afrikaans,female,south africa,20.772290,"[[-395.00208, -387.00363, -377.9013, -392.9432...","[[0.09874383, 0.25136992, 0.25506964, 0.223414...","[[0.048649844, 0.11224768, 0.10517518, 0.06032...","[[0.016398141, 0.033163264, 0.038705193, 0.063...",...,"[[-5.995369, -5.995369, -5.995369, -5.995369, ...","[[-30.363499, -29.586853, -28.244614, -23.9563...","[0.0071707093, 0.010596839, 0.00808429, 0.0093...",22050.0,"[0.01611328125, 0.0244140625, 0.033203125, 0.0...","[2950.048828125, 2971.58203125, 3068.481445312...","[1530.4223975774694, 1423.8069150707834, 1320....","[36.50567083862842, 14.012752078187578, 10.362...","[2463.8000385430314, 2273.238307315652, 2083.8...","[[0.8212306011679301, 0.8040046693991846, 0.78..."
33,40.0,"pretoria, south africa",afrikaans,male,south africa,21.961497,"[[-560.3812, -560.3812, -560.3812, -560.3812, ...","[[2.1711987e-06, 2.455684e-06, 2.3796056e-06, ...","[[2.1903074e-06, 2.4361193e-06, 2.391069e-06, ...","[[4.3559062e-10, 2.1939425e-10, 1.1125337e-09,...",...,"[[-24.149624, -24.149624, -24.149624, -24.1496...","[[-24.416653, -24.416653, -24.416653, -24.4166...","[-1.8169361e-05, -2.4786286e-05, -8.806779e-06...",22050.0,"[0.025390625, 0.041015625, 0.05029296875, 0.04...","[8990.1123046875, 9108.544921875, 9248.5107421...","[5123.089311931954, 5264.551708269, 5429.44865...","[32.56676585595034, 28.16572518059867, 24.7029...","[3348.660970144079, 3348.077664928403, 3389.43...","[[0.9168646531128495, 0.6017439302030426, 1.0,..."
34,43.0,"pretoria, transvaal, south africa",afrikaans,male,south africa,26.880000,"[[-441.16895, -439.80875, -430.03815, -406.936...","[[0.006251498, 0.005871368, 0.0032841666, 0.00...","[[0.00229122, 0.0017953095, 0.0010293978, 0.00...","[[0.0010243187, 0.0012062755, 0.00065918657, 0...",...,"[[-17.287262, -17.287262, -17.287262, -17.2872...","[[-15.318008, -15.318008, -15.318008, -15.3180...","[0.0019851814, 0.0027917789, 0.0013547958, 0.0...",22050.0,"[0.0244140625, 0.0322265625, 0.05126953125, 0....","[3079.248046875, 2627.05078125, 2616.284179687...","[1465.0935000653856, 1361.8608658509977, 1347....","[22.474867417512268, 26.549838842792123, 29.65...","[2159.5564308533862, 2036.7410293218215, 1875....","[[0.9405656123995829, 0.8434395912814198, 0.93..."
35,26.0,"pretoria, south africa",afrikaans,male,south africa,23.471202,"[[-529.75555, -529.76184, -529.74896, -529.790...","[[0.0005168101, 0.0005084191, 0.00052578124, 0...","[[0.00051670696, 0.0005085286, 0.00052552565, ...","[[5.3004323e-11, 8.718523e-11, 3.5090522e-10, ...",...,"[[-18.92363, -18.92363, -18.92363, -18.92363, ...","[[-17.007862, -17.007862, -17.007862, -17.0078...","[0.00025531906, 0.0003802524, 0.00032837203, 0...",22050.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[8182.6171875, 8322.5830078125, 8268.75, 7967....","[2836.1443828141787, 2981.4239632758927, 2943....","[31.146817865859134, 33.83682462665064, 35.432...","[3677.497426953188, 3705.7593851497895, 3685.5...","[[0.43540816224779805, 0.43058722215309736, 0...."
36,19.0,"cape town, south africa",afrikaans,male,south africa,20.252426,"[[-509.4199, -479.27243, -467.60425, -471.0367...","[[0.595527, 2.648835, 4.051913, 2.793845, 0.93...","[[0.40962306, 1.6456182, 3.0860555, 2.5644944,...","[[0.029812008, 0.13252573, 0.09091881, 0.00693...",...,"[[-7.7523117, 4.3265815, 9.7880745, 8.180036, ...","[[-30.512177, -17.553995, -20.826927, -35.1874...","[1.7647135e-05, -1.7076701e-06, -9.886354e-06,...",22050.0,"[0.0126953125, 0.01513671875, 0.0166015625, 0....","[1668.8232421875, 1604.2236328125, 1701.123046...","[808.4449193168485, 717.8543169790694, 817.630...","[20.726607494551942, 19.790129878017588, 21.93...","[1641.2152764813704, 1536.64683

In [53]:
labels = np.sort(np.unique(df['native_language'].values))
num_class = len(labels)
c2i = {}
i2c = {}
for i, c in enumerate(labels):
    c2i[c] = i
    i2c[i] = c
df["native_lang_c2i"] = np.array([c2i[x] for x in df['native_language'].values])
df["native_lang_c2i"].head()

32    0
33    0
34    0
35    0
36    0
Name: native_lang_c2i, dtype: int64

In [54]:
def setAside(df):
    train_val_data = df.sample(frac=0.8, random_state=200)
    test_data = df.drop(train_val_data.index)
    return train_val_data, test_data
svc_df_train_val,svc_df_test = setAside(df.copy())

In [56]:
from scipy.stats import skew


def get_overall_aspect_nD(ft):
    return np.hstack((np.mean(ft, axis=1), np.std(ft, axis=1), skew(ft, axis = 1), np.max(ft, axis = 1), np.median(ft, axis = 1), np.min(ft, axis = 1)))


def get_overall_aspect_1D(ft):
    return np.hstack((np.mean(ft), np.std(ft), skew(ft), np.max(ft), np.median(ft), np.min(ft)))

def get_all_features(adf):
    accumulated_features = []
    for _,row in adf.iterrows():
        mfcc_trunc = get_overall_aspect_nD(row["mfcc"])
        zcr_trunc = get_overall_aspect_1D(row["zero_crossing_rate"])
        sp_rolloff_trunc = get_overall_aspect_1D(row["spectral_rolloff"])
        sp_centroid_trunc = get_overall_aspect_1D(row["spectral_centroid"])
        sp_contrast_trunc = get_overall_aspect_1D(row["spectral_contrast"])
        sp_bandwidth_trunc = get_overall_aspect_1D(row["spectral_bandwidth"])
        accumulated_features.append(pd.Series(np.hstack((mfcc_trunc,zcr_trunc,sp_rolloff_trunc,sp_centroid_trunc,sp_contrast_trunc,sp_bandwidth_trunc))))
    return accumulated_features

accumulated_features_train = get_all_features(svc_df_train_val)
accumulated_features_test = get_all_features(svc_df_test)

In [ ]:
# y = df['native_language']
# X = df.drop(['native_language'],axis=1)
# from sklearn.model_selection import train_test_split
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)
# label_X_train = X_train.copy()
# label_X_valid = X_valid.copy()

# # Apply label encoder to each column with categorical data
# # from sklearn.preprocessing import LabelEncoder
# # label_encoder = LabelEncoder()
# object_cols = ['birthplace', 'sex', 'country']
# label_X_train.drop(columns=object_cols,inplace=True)
# label_X_valid.drop(columns=object_cols,inplace=True)
# for col in object_cols:
#     label_X_train[col] = label_encoder.fit_transform(X_train[col])
#     label_X_valid[col] = label_encoder.transform(X_valid[col])


In [ ]:
# pd.get_dummies(df, columns=['sex','country'])

In [ ]:
# import category_encoders as ce

In [ ]:
# ce_hash = ce.HashingEncoder(cols = ['country'],n_components=144)
# ce_hash.fit_transform(X, y)

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# label_encoder = LabelEncoder()
# object_cols = ['birthplace', 'native_language', 'sex', 'country']
# cat_encoded_df = df.copy()
# for col in object_cols:
#     cat_encoded_df[col] = label_encoder.fit_transform(df[col])
#     cat_encoded_df[col] = label_encoder.transform(df[col])

In [ ]:
# X.head().shape

In [57]:
svc_df_y = svc_df_train_val['native_lang_c2i']
svc_df_y_test = svc_df_test['native_lang_c2i']
# svc_df_X = svc_df.drop(columns=['birthplace','native_language','sex','country']).copy()
svc_df_X = pd.DataFrame(accumulated_features_train)
svc_df_X_test = pd.DataFrame(accumulated_features_test)
# svc_df_X = svc_df_train_val["accumulated_features"]
# svc_df_X_test = pd.DataFrame(svc_df_test["accumulated_features"])



In [59]:
# Apply scaling for PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(svc_df_X.values)
X_test_scaled = scaler.transform(svc_df_X_test)

In [62]:
# Apply PCA for dimension reduction
pca = PCA(n_components=60).fit(X_scaled)
X_pca = pca.transform(X_scaled)
X_test_pca = pca.transform(X_test_scaled)

print(sum(pca.explained_variance_ratio_)) 

0.9999999999999993


In [64]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
X_train, X_val, y_train, y_val = train_test_split(X_pca, svc_df_y, test_size = 0.2, random_state = 42, shuffle = True)

clf = SVC(kernel = 'rbf', probability=True)

clf.fit(X_train, y_train)

print(accuracy_score(clf.predict(svc_df_X_test), svc_df_y_test))

0.28337236533957844
